In [ ]:
from tools.pre_promise_competition_tool import get_pre_promise_competition_rate
result = get_pre_promise_competition_rate("강남")


    CMPET_RATE HOUSE_MANAGE_NO   HOUSE_TY MODEL_NO   PBLANC_NO REQ_CNT  \
0         4.05      2025000364  099.8274        04  2025000364     231   
1            -      2025000364  099.8274        04  2025000364       5   
2            -      2025000364  099.8274        04  2025000364      30   
3            -      2025000364  099.8274        04  2025000364       5   
4         2.41      2025000364  109.9121        05  2025000364     368   
..         ...             ...        ...      ...         ...     ...   
995          -      2025000250  084.9900A       06  2025000250     419   
996          -      2025000250  084.9900A       06  2025000250       0   
997          -      2025000250  084.9900A       06  2025000250       0   
998     703.00      2025000250  084.9400B       07  2025000250     703   
999          -      2025000250  084.9400B       07  2025000250     101   

    RESIDE_SECD RESIDE_SENM  SUBSCRPT_RANK_CODE  SUPLY_HSHLDCO  
0            01        해당지역                   